* chrome driver 다운로드  
http://chromedriver.chromium.org/downloads

In [19]:
import pandas as pd
import numpy as np
import glob
import os

from bs4 import BeautifulSoup
from selenium import webdriver

In [28]:
# *****************************
# 파라미터 설정
# *****************************
# directory 설정
readDir = 'C:/Users/jseung21/Documents/CybosPlus/data'
writeDir = 'C:/Users/jseung21/Documents/CybosPlus/data/financeReport'

In [21]:
# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome('C:/Users/jseung21/Documents/CybosPlus/chromedriver_win32/chromedriver')
driver.implicitly_wait(3)
driver.get('https://login.itooza.com/login.htm?qurl64=aHR0cDovL3d3dy5pdG9vemEuY29tLw==')

In [22]:
# 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('txtUserId').send_keys('jseung21')
driver.find_element_by_name('txtPassword').send_keys('goddns21')

# 로그인 버튼을 눌러주자.
driver.find_element_by_css_selector('div.btn-login>input').click()

C:\Users\jseung21\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.alert instead
  """Entry point for launching an IPython kernel.


In [ ]:
# 제무제표 분기 FinancialStatementsQuarter
urlFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=1&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 손익계산서 분기 IncomeStatementQuarter
urlISQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=2&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 현금흐름표 분기 CashFlowsStatementQuarter
urlCFSQ = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&mode=db&ss=10&sv=4&lsmode=1&lkmode=2&pmode=1&exmode=1&accmode=1"
# 재무비율/가치평가 연환산 financail ratio / value assessment year smooth
urlFRVAYS = "http://www.itooza.com/vclub/y10_page.php?cmp_cd={stockCode}&ss=10&sv=10"
# 재무비율/가치평가 분기
urlFRVAQ = "http://www.itooza.com/vclub/y10_page.php?ss=10&sv=10&cmp_cd={stockCode}&mode=db"
# 투자지표 연환산 Investment Index Year Smmoth
urlIIYS = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable1"
# 투자지표 분기
urlIIQ = "http://search.itooza.com/search.htm?seName={stockCode}&jl=&search_ck=&sm=&sd=&ed=&ds_de=&page=&cpv=#indexTable3"


### loop

In [ ]:
# 디렉토리안 csv 파일 가지고 오기
globbed_files = glob.glob(readDir+'/전종목.csv') #creates a list of all csv files

df_data = pd.read_csv(globbed_files[0], engine='python', encoding = 'utf8')

for index, row in df_data.iterrows():
    stockCode = row['stockCode']
    stockName = row['stockName']
#     stockCode = '000075'
#     stockName = '테스트'

    # csv 파일명
    fineName = "재무상태표_{}_{}.csv".format(stockCode, stockName)

    # url
    url = urlFSQ.format(stockCode=stockCode)
#     print('url:[{}]'.format(url))
    # 페이지 이동
    driver.get(url)
    
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # ***********************************
        # 재무항목
        # ***********************************
        _financialColumns=soup.select('div.a01 > table > tbody > tr > th')
        financialColumns = []
        for n in _financialColumns:
            financialColumns.append(n.text)

        # ***********************************
        # 분기날짜
        # ***********************************
        _dates = soup.select('table[id=y10_tb_2] > thead > tr > th')
        del _dates[0]
        dates = []
        for n in _dates:
            dates.append(n.text)

        # ***********************************
        # 재무 data
        # ***********************************
        data = soup.select('table[id=y10_tb_2] > tbody > tr > td')

        # 재무 데이터 재정리
        length = len(dates)
        j = 0
        listA = []
        for i in range (0, int(len(data)/length)):
            l = []
            for n in data[j:j+length]:
                l.append(n.text)
            listA.append(l)
            j += length

        # 재무데이터 dataFarme 생성
        df_convertData = pd.DataFrame(listA, columns=dates, index=financialColumns)

        # csv 생성
        createFile = "{}/{}".format(writeDir,fineName) 
        df_convertData.to_csv(createFile, index=True, encoding='utf-8')

        print("[{}], [{}] 생성완료".format(index, fineName))
    except Exception as ex:
        print("[{}], [{}] 생성실패 !!!".format(index, fineName))
        driver.switch_to_alert().dismiss()
#         print('에러가 발생 했습니다', ex)
        


[0], [재무상태표_000020_동화약품.csv] 생성완료
[1], [재무상태표_000030_우리은행.csv] 생성완료
[2], [재무상태표_000040_KR모터스.csv] 생성완료
[3], [재무상태표_000050_경방.csv] 생성완료
[4], [재무상태표_000060_메리츠화재.csv] 생성완료
[5], [재무상태표_000070_삼양홀딩스.csv] 생성완료
[6], [재무상태표_000075_삼양홀딩스우.csv] 생성실패 !!!


C:\Users\jseung21\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: use driver.switch_to.alert instead


[7], [재무상태표_000080_하이트진로.csv] 생성완료
[8], [재무상태표_000087_하이트진로2우B.csv] 생성실패 !!!
[9], [재무상태표_000100_유한양행.csv] 생성완료
[10], [재무상태표_000105_유한양행우.csv] 생성실패 !!!
[11], [재무상태표_000120_CJ대한통운.csv] 생성완료
[12], [재무상태표_000140_하이트진로홀딩스.csv] 생성완료
[13], [재무상태표_000145_하이트진로홀딩스우.csv] 생성실패 !!!
[14], [재무상태표_000150_두산.csv] 생성완료
[15], [재무상태표_000155_두산우.csv] 생성실패 !!!
[16], [재무상태표_000157_두산2우B.csv] 생성실패 !!!
[17], [재무상태표_000180_성창기업지주.csv] 생성완료
[18], [재무상태표_000210_대림산업.csv] 생성완료
[19], [재무상태표_000215_대림산업우.csv] 생성실패 !!!
[20], [재무상태표_000220_유유제약.csv] 생성완료
[21], [재무상태표_000225_유유제약1우.csv] 생성실패 !!!
[22], [재무상태표_000227_유유제약2우B.csv] 생성실패 !!!
[23], [재무상태표_000230_일동홀딩스.csv] 생성완료
[24], [재무상태표_000240_한국타이어월드와이드.csv] 생성완료
[25], [재무상태표_000250_삼천당제약.csv] 생성완료
[26], [재무상태표_000270_기아차.csv] 생성완료
[27], [재무상태표_000300_대유플러스.csv] 생성완료
[28], [재무상태표_000320_노루홀딩스.csv] 생성완료
[29], [재무상태표_000325_노루홀딩스우.csv] 생성실패 !!!
[30], [재무상태표_000370_한화손해보험.csv] 생성완료
[31], [재무상태표_000390_삼화페인트.csv] 생성완료
[32], [재무상태표_000400_롯데손해보험.csv] 생성완료
[33], [재무상태표_00043